# CoinMarketCap API Data Pull Project

In [3]:
# Import necessary modules from the requests library
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

# Set the API endpoint URL for the CoinMarketCap API (latest cryptocurrency listings)
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

# Define the parameters for the API request
parameters = {
    'start': '1',    # Starting point (rank) for the listings
    'limit': '5000', # Limit the number of listings to 5000
    'convert': 'USD' # Convert the prices to USD
}

# Define the headers, including the API key
headers = {
    'Accepts': 'application/json',  # Ensure the API returns JSON format
    'X-CMC_PRO_API_KEY': '6699d4f7-5975-4375-8b35-525b589736cd',  # The API key
}

# Create a session object to persist settings across requests
session = Session()

# Update the session headers with the API key and other necessary headers
session.headers.update(headers)

try:
    # Send a GET request to the API using the session object
    response = session.get(url, params=parameters)

    # Parse the JSON response text and store it in 'data'
    data = json.loads(response.text)

    # Print the parsed data (this might be large, so be cautious with this)
    print(data)

# Handle common request exceptions such as connection issues or timeouts
except (ConnectionError, Timeout, TooManyRedirects) as e:
    # Print the exception if any of the specified errors occur
    print(e)


h


In [ ]:
type(data)


In [ ]:
# Import the pandas library
import pandas as pd

# Set pandas options to display all columns and rows when printing DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [ ]:
# normalising the data to make it look readable in a data frame and adding a time stamp to see the exact time data was being pulled

df = pd.json_normalize(data['data'])

df['time_stamp'] = pd.to_datetime('now')
df

# API Call and Data Collection

In [ ]:
import os
from time import sleep

# Function to fetch and normalize the data from CoinMarketCap API
def api_auto():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

    # Define the parameters for the API request
    parameters = {
        'start': '1',    # Starting point (rank) for the listings
        'limit': '5000', # Limit the number of listings to 5000
        'convert': 'USD' # Convert the prices to USD
    }

    # Define the headers, including the API key
    headers = {
        'Accepts': 'application/json',  # Ensure the API returns JSON format
        'X-CMC_PRO_API_KEY': '6699d4f7-5975-4375-8b35-525b589736cd',  # Your API key
    }

    # Create a session object to persist settings across requests
    session = Session()

    # Update the session headers with the API key and other necessary headers
    session.headers.update(headers)

    try:
        # Send a GET request to the API using the session object
        response = session.get(url, params=parameters)

        # Parse the JSON response text and store it in 'data'
        data = json.loads(response.text)

        # Normalize the data and return it as a DataFrame
        df2 = pd.json_normalize(data['data'])

        # Add a timestamp column to the DataFrame
        df2['time_stamp'] = pd.to_datetime('now')

        # Return the DataFrame for later use
        return df2

    # Handle common request exceptions such as connection issues or timeouts
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        return None  # Return None if there's an error

# To call the API without printing the data:
df = pd.DataFrame()  # Initialize an empty DataFrame to store results

# Example loop that runs 333 times, fetches the data, and appends it to df
for i in range(333):
    df2 = api_auto()  # Fetch data from the API

    if df2 is not None:  # Only append if data was successfully fetched
        df = pd.concat([df, df2], ignore_index=True)  # Use pd.concat to append DataFrames
        print('API ran successfully')  # Print this message after each successful run

    sleep(60)  # Wait for 60 seconds between API calls

# Remove duplicates in the final DataFrame (optional, based on unique columns)
df.drop_duplicates(subset=['id'], inplace=True)  # Assuming 'id' is unique for each record


# Write data to CSV

In [ ]:
# Define the file path for the CSV
csv_file = r'C:\Users\HP\Downloads\archive (4)\APIPull.csv'

# Check if the file exists
if not os.path.isfile(csv_file):
    # If the file doesn't exist, write the DataFrame with column headers
    df.to_csv(csv_file, header=True, index=False)
    print("Data written to new CSV file.")
else:
    # If the file exists, append the data without writing the headers again
    df.to_csv(csv_file, mode='a', header=False, index=False)
    print("Data appended to existing CSV file.")


In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df